In [1]:
#!/usr/bin/env python3
import os
import sys
import tempfile
import shutil
import numpy as np
from Bio.PDB import PDBParser, MMCIFParser, PDBIO, MMCIFIO, Select
import subprocess
import os
import re

BIN = '/home/hwjang/aipd/250729/0_move/tmalign/TMalign'

AMINO_ACIDS = {
    'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS',
    'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL'
}

def detect_format(filename):
    if filename.endswith('.pdb'):
        return 'pdb'
    elif filename.endswith('.cif') or filename.endswith('.mmcif'):
        return 'mmcif'
    else:
        raise ValueError("Unsupported file format: %s" % filename)

def load_structure(filename):
    fmt = detect_format(filename)
    if fmt == 'pdb':
        parser = PDBParser(QUIET=True)
    else:
        parser = MMCIFParser(QUIET=True)
    return parser.get_structure("X", filename)

def save_structure(structure, filename):
    if filename.endswith('.pdb'):
        io = PDBIO()
        io.set_structure(structure)
        io.save(filename)
    elif filename.endswith('.cif') or filename.endswith('.mmcif'):
        io = MMCIFIO()
        io.set_structure(structure)
        io.save(filename)
    else:
        raise ValueError(f"Unsupported file format: {filename}")

class ProteinOnlySelect(Select):
    def accept_residue(self, residue):
        return residue.get_resname().strip() in AMINO_ACIDS

def extract_protein_only(infile, outfile):
    fmt = detect_format(infile)
    struct = load_structure(infile)
    if outfile.endswith('.pdb'):
        io = PDBIO()
        io.set_structure(struct)
        io.save(outfile, select=ProteinOnlySelect())
    elif outfile.endswith('.cif') or outfile.endswith('.mmcif'):
        io = MMCIFIO()
        io.set_structure(struct)
        io.save(outfile, select=ProteinOnlySelect())
    else:
        raise ValueError("Unsupported file format")

def read_matrix(fname):
    with open(fname) as f:
        lines = f.readlines()
    mat = []
    for i in range(2, 5):
        x = lines[i].strip().split()
        mat.append([float(xx) for xx in x[1:]])
    return mat

def transform_coord(coord, matrix):
    return [
        matrix[i][0] + sum(matrix[i][j+1] * coord[j] for j in range(3))
        for i in range(3)
    ]

def transform_structure(structure, matrix):
    for atom in structure.get_atoms():
        atom.set_coord(transform_coord(atom.get_coord(), matrix))

def run_tmalign(ref_fn, cmp_fn, out_prefix):
    out_matrix_fn = f"{out_prefix}_m.txt"
    cmd = [BIN, cmp_fn, ref_fn, '-m', out_matrix_fn]
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0 or not os.path.isfile(out_matrix_fn):
        raise RuntimeError(f"TM-align failed to run: {' '.join(cmd)}")

    rmsd = None
    score = None
    for line in result.stdout.splitlines():
        if "RMSD=" in line:
            m = re.search(r'RMSD=\s*([0-9.]+)', line)
            if m:
                rmsd = float(m.group(1))
        elif line.startswith("TM-score=") and score is None:
            m = re.search(r'TM-score=\s*([0-9.]+)', line)
            if m:
                score = float(m.group(1))
    if rmsd is None or score is None:
        raise RuntimeError("TM-align failed to find RMSD or TM-score in the output.")
    return out_matrix_fn, rmsd, score

def align_and_transform(ref_fn, cmp_fn, out_fn):
    tmpdir = tempfile.mkdtemp()
    try:
        ref_protein = os.path.join(tmpdir, "ref_protein." + ref_fn.split('.')[-1])
        cmp_protein = os.path.join(tmpdir, "cmp_protein." + cmp_fn.split('.')[-1])
        extract_protein_only(ref_fn, ref_protein)
        extract_protein_only(cmp_fn, cmp_protein)
        out_prefix = os.path.join(tmpdir, "tm")
        matrix_file, rmsd, score = run_tmalign(ref_protein, cmp_protein, out_prefix)
        print(f"RMSD: {rmsd}, TM-score: {score}")
        matrix = read_matrix(matrix_file)
        structure_cmp = load_structure(cmp_fn)
        transform_structure(structure_cmp, matrix)
        save_structure(structure_cmp, out_fn)
    finally:
        shutil.rmtree(tmpdir)

def align_structures_to_ref(ref_fn, cmp_fn, out_dir):
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir)
    basename = os.path.basename(ref_fn)
    out_fn = os.path.join(out_dir, basename.rsplit('.', 1)[0] + "_align.cif")
    align_and_transform(ref_fn, cmp_fn, out_fn)
    print(f"Saved: {out_fn}")

In [2]:
import os
import shutil
import tempfile
import pandas as pd
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from functools import partial

def tmalign_worker(row, model, col='lmpnn'):
    out_dir = 'tmaligned_' + model
    ref_fn = row[(col, 'path')]
    cmp_fn = row[(model, 'path')]
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    basename = os.path.basename(ref_fn)
    out_fn = os.path.join(out_dir, basename.rsplit('.', 1)[0] + "_align.cif")
    tmpdir = tempfile.mkdtemp()
    try:
        ref_protein = os.path.join(tmpdir, "ref_protein." + ref_fn.split('.')[-1])
        cmp_protein = os.path.join(tmpdir, "cmp_protein." + cmp_fn.split('.')[-1])
        extract_protein_only(ref_fn, ref_protein)
        extract_protein_only(cmp_fn, cmp_protein)
        out_prefix = os.path.join(tmpdir, "tm")
        matrix_file, rmsd, score = run_tmalign(ref_protein, cmp_protein, out_prefix)
        matrix = read_matrix(matrix_file)
        structure_cmp = load_structure(cmp_fn)
        transform_structure(structure_cmp, matrix)
        save_structure(structure_cmp, out_fn)
        out_fn = os.path.abspath(out_fn)
    finally:
        shutil.rmtree(tmpdir)
    return rmsd, score, out_fn

def process(df, model, col='lmpnn', pool_size=cpu_count()):
    rows = [row for _, row in df.iterrows()]
    worker = partial(tmalign_worker, model=model, col=col)
    with Pool(pool_size) as pool:
        results = list(tqdm(pool.imap(worker, rows), total=len(rows)))
    tm_rmsd, tm_score, tm_path = zip(*results)
    df[(model, 'tm_rmsd')] = tm_rmsd
    df[(model, 'tm_score')] = tm_score
    df[(model, 'tm_path')] = tm_path
    return df

path = 'rmsd_1st.parquet'
df = pd.read_parquet(path)

df = process(df, model='af3', col='lmpnn', pool_size=8)
df = process(df, model='boltz', col='lmpnn', pool_size=8)

100%|██████████| 36/36 [00:01<00:00, 29.87it/s]


In [3]:
df.to_parquet('rmsd_2nd.parquet')
df

diffusion                           lmpnn             \
                      id     batch                   tag        ddg   
0    result_7_packed_3_1  pht_demo   result_7_packed_3_1 -27.749462   
1   result_61_packed_8_1  pht_demo  result_61_packed_8_1 -26.640675   
2    result_7_packed_8_1  pht_demo   result_7_packed_8_1 -25.789537   
3    result_7_packed_2_1  pht_demo   result_7_packed_2_1 -25.547806   
4   result_16_packed_4_1  pht_demo  result_16_packed_4_1 -25.318375   
5   result_16_packed_7_1  pht_demo  result_16_packed_7_1 -24.987370   
6    result_7_packed_4_1  pht_demo   result_7_packed_4_1 -23.823254   
7   result_29_packed_7_1  pht_demo  result_29_packed_7_1 -22.682110   
8   result_59_packed_1_1  pht_demo  result_59_packed_1_1 -22.405994   
9   result_59_packed_6_1  pht_demo  result_59_packed_6_1 -21.156240   
10   result_7_packed_1_1  pht_demo   result_7_packed_1_1 -21.103624   
11  result_24_packed_8_1  pht_demo  result_24_packed_8_1 -20.956982   
12  result_24_packed_7_1  pht_demo  result_24_packed_7_1 -20.956400   
13  result_59_packed_3_1  pht_demo  result_59_packed_3_1 -20.733452   
14  result_55_packed_1_1  pht_demo  result_55_packed_1_1 -20.355864   
15  result_59_packed_8_1  pht_demo  result_59_packed_8_1 -20.324280   
16  result_24_packed_3_1  pht_demo  result_24_packed_3_1 -20.292067   
17  result_24_packed_6_1  pht_demo  result_24_packed_6_1 -20.236935   
18  result_59_packed_7_1  pht_demo  result_59_packed_7_1 -20.159588   
19  result_55_packed_2_1  pht_demo  result_55_packed_2_1 -19.940771   
20  result_24_packed_4_1  pht_demo  result_24_packed_4_1 -19.784569   
21  result_24_packed_5_1  pht_demo  result_24_packed_5_1 -19.718063   
22  result_10_packed_4_1  pht_demo  result_10_packed_4_1 -19.292791   
23  result_29_packed_3_1  pht_demo  result_29_packed_3_1 -19.285507   
24  result_55_packed_4_1  pht_demo  result_55_packed_4_1 -19.126881   
25  result_24_packed_1_1  pht_demo  result_24_packed_1_1 -18.947884   
26  result_55_packed_5_1  pht_demo  result_55_packed_5_1 -18.944952   
27  result_24_packed_2_1  pht_demo  result_24_packed_2_1 -18.919436   
28  result_59_packed_2_1  pht_demo  result_59_packed_2_1 -18.819977   
29  result_59_packed_5_1  pht_demo  result_59_packed_5_1 -18.560551   
30  result_16_packed_2_1  pht_demo  result_16_packed_2_1 -18.512140   
31  result_55_packed_7_1  pht_demo  result_55_packed_7_1 -18.505692   
32   result_4_packed_3_1  pht_demo   result_4_packed_3_1 -18.452574   
33  result_15_packed_7_1  pht_demo  result_15_packed_7_1 -18.338469   
34  result_29_packed_6_1  pht_demo  result_29_packed_6_1 -18.066357   
35   result_5_packed_6_1  pht_demo   result_5_packed_6_1 -18.049738   

                                           \
        fa_rep res_totalscore  totalscore   
0    95.805969      -1.583578 -199.530838   
1   104.494186      -1.441779 -181.664185   
2    98.276131      -1.570810 -197.922104   
3   102.521637      -1.610180 -202.882706   
4    91.409203      -1.448796 -160.816406   
5    88.052048      -1.478883 -164.156021   
6   103.836670      -1.429071 -180.062973   
7    75.388725      -1.675448 -177.597519   
8    70.000290      -1.624849 -180.358276   
9    66.255737      -1.786232 -198.271744   
10   97.976486      -1.563029 -196.941681   
11   73.818649      -1.721328 -194.510040   
12   75.090805      -1.810861 -204.627274   
13   66.947762      -1.525246 -169.302292   
14   69.711243      -1.708151 -182.772202   
15   65.809105      -1.677221 -186.171524   
16   68.647247      -1.709947 -193.223969   
17   76.706245      -1.489178 -168.277100   
18   69.610352      -1.726851 -191.680466   
19   66.307579      -1.638223 -175.289825   
20   75.684486      -1.614937 -182.487900   
21   73.074364      -1.775147 -200.591599   
22   83.306686      -1.661632 -177.794617   
23   81.779648      -1.521618 -161.291504   
24   67.392075      -1.756452 -187.940399   
25   74.767509      -1.562683 -176.583221   
26   71.573624      -1.511618 -161.743149   
27   75.802